In [1]:
import re
import pandas as pd

from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.translate.bleu_score import sentence_bleu

from src.config import mturk_simjudgement_file, mturk_substitution_rating_scores

import warnings
warnings.filterwarnings('ignore')

In [2]:
tokenizer = RegexpTokenizer(r'\w+')
def clean_text(s):
    try:
        return ' '.join(tokenizer.tokenize(re.sub(r'[^a-zA-Z ]',r'', s).lower()))
    except:
        print(s)
        return ''


def get_weighted_scores(single_cands, single_refs, weights=(0.25, 0.25, 0.25, 0.25)): 
    hypothesis = word_tokenize(clean_text(single_cands))
    reference = word_tokenize(clean_text(single_refs))
    return sentence_bleu([reference], hypothesis, weights=weights)

In [3]:
df = pd.read_csv(mturk_simjudgement_file)

weight_scheme = {'BLEU-1':(1,0,0,0), 'BLEU-2':(0.5, 0.5, 0, 0)}
for m, w in weight_scheme.items():
    def weighted_scores(line):
        single_cands = line['rec_item']
        single_ref = line['gt_item']
        return get_weighted_scores(single_cands, single_ref, weights=w)

    df[m]= df.apply(weighted_scores, axis=1)

filename = mturk_substitution_rating_scores['BLEU']
df.to_csv(filename, index=True)

In [4]:
df.corr().iloc[3:4, 4:]

,BLEU-1,BLEU-2
SimJudgement,0.42495,0.369848
